# 01 - Data Loading and Preprocessing

This notebook loads all raw datasets, performs cleaning and preprocessing, and exports processed data for use in subsequent analyses.

## Contents
1. Setup and Imports
2. Load Earthquake Data
3. Load Supporting Datasets
4. Data Cleaning
5. Feature Engineering
6. Data Merging
7. Export Processed Data

## 1. Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src to path
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Import project utilities
from config import *
from geo_utils import haversine_distance
from seismology import calculate_energy, calculate_moment

print(f"Base path: {BASE_PATH}")
print(f"Data raw: {DATA_RAW}")
print(f"Data processed: {DATA_PROCESSED}")

Base path: /Users/boraesen/Desktop/Semester 7/Stat495/stat495project
Data raw: /Users/boraesen/Desktop/Semester 7/Stat495/stat495project/data/raw
Data processed: /Users/boraesen/Desktop/Semester 7/Stat495/stat495project/data/processed


## 2. Load Earthquake Data

In [2]:
# Load main earthquake dataset
eq_df = pd.read_csv(EARTHQUAKE_DATA)
print(f"Loaded {len(eq_df):,} earthquake records")
print(f"\nColumns: {eq_df.columns.tolist()}")
eq_df.head()

Loaded 537,378 earthquake records

Columns: ['rms', 'eventID', 'location', 'latitude', 'longitude', 'depth', 'type', 'magnitude', 'country', 'province', 'district', 'neighborhood', 'date', 'isEventUpdate', 'lastUpdateDate']


,rms,eventID,location,latitude,longitude,depth,type,magnitude,country,province,district,neighborhood,date,isEventUpdate,lastUpdateDate
0,0.0,237966,Kandıra (Kocaeli),41.0,30.0,7.0,Md,3.0,Türkiye,Kocaeli,Kandıra,Hacışeyh,1990-01-03T13:30:14,False,NaN
1,0.0,237967,Pazaryeri (Bilecik),40.0,30.0,1.0,Md,3.0,Türkiye,Bilecik,Pazaryeri,Demirköy,1990-01-04T11:32:14,False,NaN
2,0.0,237968,Mengen (Bolu),41.0,32.0,7.0,Md,3.0,Türkiye,Bolu,Mengen,Çubuk,1990-01-06T12:59:39,False,NaN
3,1.0,237969,Kulu (Konya),39.0,33.0,12.0,Md,3.0,Türkiye,Konya,Kulu,Celep,1990-01-06T14:08:16,False,NaN
4,0.0,237970,Altındağ (Ankara),40.0,33.0,1.0,Md,3.0,Türkiye,Ankara,Altındağ,Tatlar,1990-01-07T04:46:31,False,NaN


In [3]:
# Check data types and missing values
print("Data Types:")
print(eq_df.dtypes)
print("\nMissing Values:")
print(eq_df.isnull().sum())

Data Types:
rms               float64
eventID             int64
location           object
latitude          float64
longitude         float64
depth             float64
type               object
magnitude         float64
country            object
province           object
district           object
neighborhood       object
date               object
isEventUpdate        bool
lastUpdateDate     object
dtype: object

Missing Values:
rms                    0
eventID                0
location              14
latitude               0
longitude              0
depth                  0
type                   0
magnitude              0
country             6905
province           16238
district           16238
neighborhood      106851
date                   0
isEventUpdate          0
lastUpdateDate    535204
dtype: int64


In [4]:
# Parse dates (handle mixed format)
eq_df['date'] = pd.to_datetime(eq_df['date'], format='mixed')

# Verify date range
print(f"Date range: {eq_df['date'].min()} to {eq_df['date'].max()}")

Date range: 1990-01-03 13:30:14 to 2025-11-20 16:51:56


## 3. Load Supporting Datasets

In [5]:
# Load soil classification data (provinces)
soil_province_df = pd.read_csv(SOIL_PROVINCE_DATA)
print(f"Soil Province Data: {len(soil_province_df)} provinces")
print(f"Columns: {soil_province_df.columns.tolist()}")
soil_province_df.head()

Soil Province Data: 81 provinces
Columns: ['province_id', 'province_name', 'region', 'lat', 'lon', 'elevation_m', 'vs30_estimated', 'soil_class', 'soil_class_desc', 'vs30_range_min', 'vs30_range_max', 'liquefaction_risk', 'liquefaction_risk_tr', 'amplification_factor', 'seismic_hazard', 'seismic_hazard_tr', 'geology_type', 'tectonic_setting']


,province_id,province_name,region,lat,lon,elevation_m,vs30_estimated,soil_class,soil_class_desc,vs30_range_min,vs30_range_max,liquefaction_risk,liquefaction_risk_tr,amplification_factor,seismic_hazard,seismic_hazard_tr,geology_type,tectonic_setting
0,1,Adana,Akdeniz,37.0000,35.3213,23,210,ZC,Dense soil (sıkı zemin),160,260,High,Yüksek,1.90,High,Yüksek,Limestone/alluvium,Moderate
1,2,Adıyaman,Güneydoğu Anadolu,37.7648,38.2786,672,398,ZB,Soft rock (yumuşak kaya),348,448,Very Low,Çok Düşük,1.38,Very High,Çok Yüksek,Sedimentary/limestone,Active EAF
2,3,Afyonkarahisar,Ege,38.7507,30.5567,1021,345,ZC,Dense soil (sıkı zemin),295,395,Medium,Orta,1.48,High,Yüksek,Graben sediments,Active extension
3,4,Ağrı,Doğu Anadolu,39.7191,43.0503,1640,570,ZB,Soft rock (yumuşak kaya),520,620,Very Low,Çok Düşük,1.15,High,Yüksek,Volcanic/metamorphic,Active collision
4,5,Amasya,Karadeniz,40.6499,35.8353,412,402,ZB,Soft rock (yumuşak kaya),352,452,Very Low,Çok Düşük,1.37,Medium,Orta,Metamorphic/volcanic,Stable


In [6]:
# Load soil classification data (districts)
soil_district_df = pd.read_csv(SOIL_DISTRICT_DATA)
print(f"Soil District Data: {len(soil_district_df)} districts")
soil_district_df.head()

Soil District Data: 973 districts


,district_id,province_name,district_name,region,lat,lon,elevation_m,vs30_estimated,soil_class,liquefaction_risk,seismic_hazard,amplification_factor
0,1,Adana,Aladağ,Akdeniz,37.0000,35.5232,0,223,ZC,Medium,High,1.85
1,2,Adana,Ceyhan,Akdeniz,37.0709,35.5112,0,195,ZC,Medium,High,1.97
2,3,Adana,Çukurova,Akdeniz,37.1411,35.4725,185,234,ZC,Medium,High,1.80
3,4,Adana,Feke,Akdeniz,37.0983,35.3583,0,226,ZC,Medium,High,1.83
4,5,Adana,İmamoğlu,Akdeniz,37.1689,35.2987,178,224,ZC,Medium,High,1.84


In [7]:
# Load fault data
fault_df = pd.read_csv(FAULT_DATA)
print(f"Fault Data: {len(fault_df)} fault segments")
print(f"Columns: {fault_df.columns.tolist()}")
fault_df.head()

Fault Data: 20 fault segments
Columns: ['fault_id', 'fault_name', 'fault_name_en', 'fault_zone', 'segment', 'fault_type', 'fault_type_en', 'length_km', 'slip_rate_mm_yr', 'last_major_earthquake', 'last_eq_magnitude', 'lat_start', 'lon_start', 'lat_end', 'lon_end', 'max_expected_magnitude', 'recurrence_interval_years', 'activity_class', 'seismic_gap', 'data_source', 'coordinate_system', 'last_update']


,fault_id,fault_name,fault_name_en,fault_zone,segment,fault_type,fault_type_en,length_km,slip_rate_mm_yr,last_major_earthquake,...,lon_start,lat_end,lon_end,max_expected_magnitude,recurrence_interval_years,activity_class,seismic_gap,data_source,coordinate_system,last_update
0,NAF-001,Kuzey Anadolu Fayı - Marmara Segmenti,North Anatolian Fault - Marmara Segment,Kuzey Anadolu Fay Zonu,Marmara,Sağ yanal doğrultu atımlı,Right-lateral strike-slip,150,24,1999-08-17,...,29.86,40.45,27.50,7.4,250,A,True,MTA Türkiye Diri Fay Haritası 2013 + Literature,WGS84,2023-06-01
1,NAF-002,Kuzey Anadolu Fayı - Düzce Segmenti,North Anatolian Fault - Düzce Segment,Kuzey Anadolu Fay Zonu,Düzce,Sağ yanal doğrultu atımlı,Right-lateral strike-slip,40,22,1999-11-12,...,31.21,40.74,30.38,7.2,200,A,False,MTA Türkiye Diri Fay Haritası 2013 + Literature,WGS84,2023-06-01
2,NAF-003,Kuzey Anadolu Fayı - Erzincan Segmenti,North Anatolian Fault - Erzincan Segment,Kuzey Anadolu Fay Zonu,Erzincan,Sağ yanal doğrultu atımlı,Right-lateral strike-slip,120,20,1992-03-13,...,39.65,39.55,38.10,7.5,300,A,False,MTA Türkiye Diri Fay Haritası 2013 + Literature,WGS84,2023-06-01
3,NAF-004,Kuzey Anadolu Fayı - Niksar Segmenti,North Anatolian Fault - Niksar Segment,Kuzey Anadolu Fay Zonu,Niksar,Sağ yanal doğrultu atımlı,Right-lateral strike-slip,85,18,1942-12-20,...,37.20,40.55,35.80,7.3,280,A,True,MTA Türkiye Diri Fay Haritası 2013 + Literature,WGS84,2023-06-01
4,NAF-005,Kuzey Anadolu Fayı - Tosya Segmenti,North Anatolian Fault - Tosya Segment,Kuzey Anadolu Fay Zonu,Tosya,Sağ yanal doğrultu atımlı,Right-lateral strike-slip,95,20,1943-11-26,...,34.20,40.60,33.00,7.4,260,A,True,MTA Türkiye Diri Fay Haritası 2013 + Literature,WGS84,2023-06-01


In [8]:
# Load GPS velocity data
gps_df = pd.read_csv(GPS_DATA)
print(f"GPS Data: {len(gps_df)} stations")
gps_df.head()

GPS Data: 17 stations


,station_id,station_name,lat,lon,velocity_east_mm_yr,velocity_north_mm_yr,velocity_total_mm_yr,azimuth_deg,uncertainty_mm_yr,plate,region,reference_frame,data_source,measurement_period
0,ANKR,Ankara,39.89,32.76,-3.2,-15.8,16.1,191,1.5,Anatolia,Central Anatolia,Eurasia-fixed (ITRF2014),"GPS measurements (McClusky et al. 2000, Reilin...",1988-2020
1,ISTA,Istanbul,41.10,29.02,-7.5,-19.2,20.6,201,1.2,Anatolia,Marmara,Eurasia-fixed (ITRF2014),"GPS measurements (McClusky et al. 2000, Reilin...",1988-2020
2,IZMI,İzmir,38.39,27.08,-12.5,-22.3,25.6,209,1.3,Anatolia,Aegean,Eurasia-fixed (ITRF2014),"GPS measurements (McClusky et al. 2000, Reilin...",1988-2020
3,ANTE,Antakya,36.20,36.16,2.1,-12.5,12.7,170,1.8,Arabia/Anatolia boundary,Hatay,Eurasia-fixed (ITRF2014),"GPS measurements (McClusky et al. 2000, Reilin...",1988-2020
4,TRAB,Trabzon,41.00,39.72,-1.5,-11.2,11.3,188,1.6,Anatolia,Black Sea,Eurasia-fixed (ITRF2014),"GPS measurements (McClusky et al. 2000, Reilin...",1988-2020


In [9]:
# Load moon daily data
moon_df = pd.read_csv(MOON_DATA)
moon_df['date'] = pd.to_datetime(moon_df['date'])
print(f"Moon Data: {len(moon_df)} days")
print(f"Columns: {moon_df.columns.tolist()}")

# Rename columns for consistency
moon_df = moon_df.rename(columns={
    'moon_illumination_pct': 'illumination',
    'moon_phase_angle': 'phase_angle'
})

# Calculate moon phase from phase angle
def get_moon_phase(phase_angle):
    """Determine moon phase from phase angle (0-180 degrees)."""
    if phase_angle < 45:
        return 'New Moon'
    elif phase_angle < 90:
        return 'Waxing'
    elif phase_angle < 135:
        return 'Full Moon'
    else:
        return 'Waning'

moon_df['moon_phase'] = moon_df['phase_angle'].apply(get_moon_phase)

# Calculate moon age (days since new moon, approximation)
moon_df['moon_age'] = (moon_df['phase_angle'] / 180 * 14.76).round(1)

print("\nMoon phase distribution:")
print(moon_df['moon_phase'].value_counts())
moon_df.head()

Moon Data: 13149 days
Columns: ['date', 'moon_distance_au', 'moon_illumination_pct', 'moon_phase_angle']

Moon phase distribution:
moon_phase
Waning       7654
New Moon     2961
Waxing       1368
Full Moon    1166
Name: count, dtype: int64


,date,moon_distance_au,illumination,phase_angle,moon_phase,moon_age
0,1990-01-01,0.002545,15.496739,55.788262,Waxing,4.6
1,1990-01-02,0.002525,24.136755,86.892318,Waxing,7.1
2,1990-01-03,0.002507,34.154224,122.955208,Full Moon,10.1
3,1990-01-04,0.002489,45.098438,162.354378,Waning,13.3
4,1990-01-05,0.002475,56.435455,203.167639,Waning,16.7


In [10]:
# Load moon phases data
moon_phases_df = pd.read_csv(MOON_PHASES_DATA)
moon_phases_df['date'] = pd.to_datetime(moon_phases_df['date'])
print(f"Moon Phases Data: {len(moon_phases_df)} phase events")
moon_phases_df.head()

Moon Phases Data: 1781 phase events


,date,phase,phase_code,illumination,year,month,day
0,1990-01-04 10:40:20.976507,First Quarter,1,50,1990,1,4
1,1990-01-11 04:56:49.964128,Full Moon,2,100,1990,1,11
2,1990-01-18 21:17:17.668993,Last Quarter,3,50,1990,1,18
3,1990-01-26 19:20:02.225230,New Moon,0,0,1990,1,26
4,1990-02-02 18:32:26.162860,First Quarter,1,50,1990,2,2


In [11]:
# Load atmospheric pressure data
pressure_df = pd.read_csv(PRESSURE_DATA)
pressure_df['date'] = pd.to_datetime(pressure_df['date'])
print(f"Pressure Data: {len(pressure_df)} records")
print(f"Columns: {pressure_df.columns.tolist()}")

# Rename column for consistency
pressure_df = pressure_df.rename(columns={'pressure_change_hpa': 'pressure_change'})
pressure_df.head()

Pressure Data: 13149 records
Columns: ['date', 'pressure_hpa', 'pressure_change_hpa', 'year', 'month', 'day', 'day_of_year', 'location', 'lat', 'lon', 'elevation_m', 'data_type']


,date,pressure_hpa,pressure_change,year,month,day,day_of_year,location,lat,lon,elevation_m,data_type
0,1990-01-01,908.0,7.3,1990,1,1,1,"Ankara, Turkey (representative)",39.93,32.86,938,Synthetic (ERA5-based climatology)
1,1990-01-02,904.9,-3.2,1990,1,2,2,"Ankara, Turkey (representative)",39.93,32.86,938,Synthetic (ERA5-based climatology)
2,1990-01-03,888.9,-15.9,1990,1,3,3,"Ankara, Turkey (representative)",39.93,32.86,938,Synthetic (ERA5-based climatology)
3,1990-01-04,909.4,20.5,1990,1,4,4,"Ankara, Turkey (representative)",39.93,32.86,938,Synthetic (ERA5-based climatology)
4,1990-01-05,902.8,-6.7,1990,1,5,5,"Ankara, Turkey (representative)",39.93,32.86,938,Synthetic (ERA5-based climatology)


In [12]:
# Load population density data
population_df = pd.read_excel(POPULATION_DATA, skiprows=1)
population_df.columns = ['year', 'province', 'population_density']

# Clean and convert types
population_df['year'] = pd.to_numeric(population_df['year'], errors='coerce')
population_df['population_density'] = pd.to_numeric(population_df['population_density'], errors='coerce')
population_df['province'] = population_df['province'].str.strip().str.upper()

# Standardize province names for merging
population_df['province_std'] = population_df['province'].apply(
    lambda x: str(x).replace('İ', 'I').replace('Ş', 'S').replace('Ğ', 'G').replace('Ü', 'U').replace('Ö', 'O').replace('Ç', 'C') if pd.notna(x) else 'Unknown'
)

print(f"Population Data: {len(population_df)} records")
print(f"Years available: {sorted(population_df['year'].dropna().unique().astype(int))}")
print(f"Unique provinces: {population_df['province'].nunique()}")
population_df.head(10)

Population Data: 2327 records
Years available: [np.int64(1935), np.int64(1940), np.int64(1945), np.int64(1950), np.int64(1955), np.int64(1960), np.int64(1965), np.int64(1970), np.int64(1975), np.int64(1980), np.int64(1985), np.int64(1990), np.int64(2000), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
Unique provinces: 81


,year,province,population_density,province_std
0,2024,ADANA,169.4,ADANA
1,2024,ADIYAMAN,81.8,ADIYAMAN
2,2024,AFYONKARAHİSAR,54.6,AFYONKARAHISAR
3,2024,AĞRI,44.3,AGRI
4,2024,AKSARAY,60.9,AKSARAY
5,2024,AMASYA,60.6,AMASYA
6,2024,ANKARA,236.0,ANKARA
7,2024,ANTALYA,132.5,ANTALYA
8,2024,ARDAHAN,18.8,ARDAHAN
9,2024,ARTVİN,23.6,ARTVIN


## 4. Data Cleaning

In [13]:
# Remove duplicates
initial_count = len(eq_df)
eq_df = eq_df.drop_duplicates()
print(f"Removed {initial_count - len(eq_df)} duplicate records")

# Remove invalid coordinates
initial_count = len(eq_df)
eq_df = eq_df[
    (eq_df['latitude'] >= TURKEY_BOUNDS['lat_min']) & 
    (eq_df['latitude'] <= TURKEY_BOUNDS['lat_max']) &
    (eq_df['longitude'] >= TURKEY_BOUNDS['lon_min']) & 
    (eq_df['longitude'] <= TURKEY_BOUNDS['lon_max'])
]
print(f"Removed {initial_count - len(eq_df)} records outside Turkey bounds")

# Remove invalid magnitudes
initial_count = len(eq_df)
eq_df = eq_df[eq_df['magnitude'] > 0]
print(f"Removed {initial_count - len(eq_df)} records with invalid magnitude")

# Remove invalid depths
initial_count = len(eq_df)
eq_df = eq_df[(eq_df['depth'] >= 0) & (eq_df['depth'] <= 700)]  # Max realistic depth
print(f"Removed {initial_count - len(eq_df)} records with invalid depth")

print(f"\nFinal record count: {len(eq_df):,}")

Removed 0 duplicate records
Removed 12113 records outside Turkey bounds


Removed 311 records with invalid magnitude
Removed 2 records with invalid depth

Final record count: 524,952


In [14]:
# Handle missing province/district values
eq_df['province'] = eq_df['province'].fillna('Unknown')
eq_df['district'] = eq_df['district'].fillna('Unknown')

print("Missing value counts after cleaning:")
print(eq_df.isnull().sum())

Missing value counts after cleaning:
rms                    0
eventID                0
location              13
latitude               0
longitude              0
depth                  0
type                   0
magnitude              0
country             2785
province               0
district               0
neighborhood       98590
date                   0
isEventUpdate          0
lastUpdateDate    522872
dtype: int64


## 5. Feature Engineering

In [15]:
# Classification: Earthquake (M >= 4.0) vs Tremor (M < 4.0)
eq_df['category'] = np.where(
    eq_df['magnitude'] >= MAGNITUDE_THRESHOLD, 
    'Earthquake', 
    'Tremor'
)

print("Category distribution:")
print(eq_df['category'].value_counts())
print(f"\nPercentage of Earthquakes (M >= {MAGNITUDE_THRESHOLD}): {(eq_df['category'] == 'Earthquake').mean()*100:.2f}%")

Category distribution:
category
Tremor        520849
Earthquake      4103
Name: count, dtype: int64

Percentage of Earthquakes (M >= 4.0): 0.78%


In [16]:
# Extract temporal features
eq_df['year'] = eq_df['date'].dt.year
eq_df['month'] = eq_df['date'].dt.month
eq_df['day'] = eq_df['date'].dt.day
eq_df['hour'] = eq_df['date'].dt.hour
eq_df['day_of_week'] = eq_df['date'].dt.dayofweek
eq_df['day_of_year'] = eq_df['date'].dt.dayofyear
eq_df['week_of_year'] = eq_df['date'].dt.isocalendar().week
eq_df['quarter'] = eq_df['date'].dt.quarter

# Date only (for merging with daily data)
eq_df['date_only'] = eq_df['date'].dt.date

print("Temporal features added:")
print(eq_df[['date', 'year', 'month', 'day', 'hour', 'day_of_week']].head())

Temporal features added:
                 date  year  month  day  hour  day_of_week
0 1990-01-03 13:30:14  1990      1    3    13            2
1 1990-01-04 11:32:14  1990      1    4    11            3
2 1990-01-06 12:59:39  1990      1    6    12            5
3 1990-01-06 14:08:16  1990      1    6    14            5
4 1990-01-07 04:46:31  1990      1    7     4            6


In [17]:
# Calculate seismic energy and moment
eq_df['energy_joules'] = calculate_energy(eq_df['magnitude'])
eq_df['moment_dyne_cm'] = calculate_moment(eq_df['magnitude'])

# Log-transformed values for visualization
eq_df['log_energy'] = np.log10(eq_df['energy_joules'])
eq_df['log_moment'] = np.log10(eq_df['moment_dyne_cm'])

print("Energy statistics:")
print(eq_df[['magnitude', 'energy_joules', 'log_energy']].describe())

Energy statistics:
           magnitude  energy_joules     log_energy
count  524952.000000   5.249520e+05  524952.000000
mean        1.980904   1.154625e+11       7.771357
std         0.708176   3.994746e+13       1.062264
min         0.200000   1.258925e+05       5.100000
25%         1.400000   7.943282e+06       6.900000
50%         1.900000   4.466836e+07       7.650000
75%         2.500000   3.548134e+08       8.550000
max         7.700000   2.238721e+16      16.350000


In [18]:
# Depth classification
def classify_depth(depth):
    if depth <= 10:
        return 'Very Shallow (0-10 km)'
    elif depth <= 30:
        return 'Shallow (10-30 km)'
    elif depth <= 70:
        return 'Intermediate (30-70 km)'
    else:
        return 'Deep (>70 km)'

eq_df['depth_class'] = eq_df['depth'].apply(classify_depth)

print("Depth classification distribution:")
print(eq_df['depth_class'].value_counts())

Depth classification distribution:
depth_class
Very Shallow (0-10 km)     457482
Shallow (10-30 km)          62832
Intermediate (30-70 km)      3967
Deep (>70 km)                 671
Name: count, dtype: int64


In [19]:
# Magnitude bins for analysis
magnitude_bins = [0, 2, 3, 4, 5, 6, 7, 10]
magnitude_labels = ['0-2', '2-3', '3-4', '4-5', '5-6', '6-7', '7+']
eq_df['magnitude_bin'] = pd.cut(eq_df['magnitude'], bins=magnitude_bins, labels=magnitude_labels)

print("Magnitude bin distribution:")
print(eq_df['magnitude_bin'].value_counts().sort_index())

Magnitude bin distribution:
magnitude_bin
0-2    304627
2-3    184243
3-4     32975
4-5      2866
5-6       220
6-7        17
7+          4
Name: count, dtype: int64


## 6. Calculate Distance to Nearest Fault

In [20]:
# Prepare fault midpoints for distance calculation
# Actual column names: lat_start, lon_start, lat_end, lon_end
fault_midpoints = np.column_stack([
    (fault_df['lat_start'].values + fault_df['lat_end'].values) / 2,
    (fault_df['lon_start'].values + fault_df['lon_end'].values) / 2
])

print(f"Prepared {len(fault_midpoints)} fault midpoints for distance calculation")

Prepared 20 fault midpoints for distance calculation


In [21]:
def vectorized_fault_distance(eq_lats, eq_lons, fault_coords):
    """Vectorized calculation of minimum distance to faults."""
    distances = []
    
    for lat, lon in zip(eq_lats, eq_lons):
        dists = [
            haversine_distance(lat, lon, f_lat, f_lon) 
            for f_lat, f_lon in fault_coords
        ]
        distances.append(min(dists))
    
    return np.array(distances)

# Calculate for significant earthquakes only (M >= 4.0) to save time
significant_eq = eq_df[eq_df['magnitude'] >= 4.0].copy()
print(f"Calculating fault distances for {len(significant_eq):,} significant earthquakes...")

# Calculate distances
significant_eq['fault_distance_km'] = vectorized_fault_distance(
    significant_eq['latitude'].values,
    significant_eq['longitude'].values,
    fault_midpoints
)

print(f"\nFault distance statistics (M >= 4.0):")
print(significant_eq['fault_distance_km'].describe())

Calculating fault distances for 4,103 significant earthquakes...



Fault distance statistics (M >= 4.0):
count    4103.000000
mean       96.306320
std        74.947298
min         0.974181
25%        43.321753
50%        70.949008
75%       126.935220
max       449.722979
Name: fault_distance_km, dtype: float64


In [22]:
# Merge fault distance back to main dataframe
eq_df = eq_df.merge(
    significant_eq[['eventID', 'fault_distance_km']], 
    on='eventID', 
    how='left'
)

print(f"Records with fault distance: {eq_df['fault_distance_km'].notna().sum():,}")

Records with fault distance: 4,103


## 7. Merge with Supporting Data

In [23]:
# Standardize province names for merging
def standardize_province_name(name):
    """Standardize Turkish province names."""
    if pd.isna(name):
        return 'Unknown'
    name = str(name).strip().upper()
    # Handle common variations
    name = name.replace('İ', 'I').replace('Ş', 'S').replace('Ğ', 'G')
    name = name.replace('Ü', 'U').replace('Ö', 'O').replace('Ç', 'C')
    return name

eq_df['province_std'] = eq_df['province'].apply(standardize_province_name)

# Standardize soil province data - note column is province_name
soil_province_df['province_std'] = soil_province_df['province_name'].apply(standardize_province_name)

# Rename columns for consistency
soil_province_df = soil_province_df.rename(columns={'vs30_estimated': 'vs30'})

In [24]:
# Merge with soil data
eq_with_soil = eq_df.merge(
    soil_province_df[['province_std', 'soil_class', 'vs30', 'liquefaction_risk', 'seismic_hazard']], 
    on='province_std', 
    how='left'
)

print(f"Records with soil data: {eq_with_soil['soil_class'].notna().sum():,}")
print(f"\nSoil class distribution:")
print(eq_with_soil['soil_class'].value_counts())

Records with soil data: 515,089

Soil class distribution:
soil_class
ZC    283216
ZB    186701
ZE     45172
Name: count, dtype: int64


In [25]:
# Merge with moon phase data
moon_df['date_only'] = moon_df['date'].dt.date

eq_with_all = eq_with_soil.merge(
    moon_df[['date_only', 'moon_phase', 'illumination', 'moon_age']],
    on='date_only',
    how='left'
)

print(f"Records with moon data: {eq_with_all['moon_phase'].notna().sum():,}")

Records with moon data: 524,952


In [26]:
# Merge with pressure data
pressure_df['date_only'] = pressure_df['date'].dt.date

eq_final = eq_with_all.merge(
    pressure_df[['date_only', 'pressure_hpa', 'pressure_change']],
    on='date_only',
    how='left'
)

print(f"Records with pressure data: {eq_final['pressure_hpa'].notna().sum():,}")

Records with pressure data: 524,952


In [27]:
# Merge with population density data
# For years not in population data, use the nearest available year
def get_nearest_pop_year(year):
    """Map earthquake year to nearest available population year."""
    pop_years = sorted(population_df['year'].dropna().unique())
    if year in pop_years:
        return year
    # Find nearest year
    nearest = min(pop_years, key=lambda x: abs(x - year))
    return nearest

eq_final['pop_year'] = eq_final['year'].apply(get_nearest_pop_year)

# Merge population density
eq_final = eq_final.merge(
    population_df[['year', 'province_std', 'population_density']],
    left_on=['pop_year', 'province_std'],
    right_on=['year', 'province_std'],
    how='left',
    suffixes=('', '_pop')
)

# Clean up columns
eq_final = eq_final.drop(columns=['year_pop', 'pop_year'], errors='ignore')

print(f"Records with population density data: {eq_final['population_density'].notna().sum():,}")
print(f"\nPopulation density statistics:")
print(eq_final['population_density'].describe())

Records with population density data: 514,965

Population density statistics:
count    514965.000000
mean        120.590758
std         217.496497
min          10.300000
25%          56.300000
50%          77.800000
75%         110.200000
max        3061.600000
Name: population_density, dtype: float64


In [28]:
# Final dataset summary
print("=" * 60)
print("FINAL DATASET SUMMARY")
print("=" * 60)
print(f"Total records: {len(eq_final):,}")
print(f"Date range: {eq_final['date'].min()} to {eq_final['date'].max()}")
print(f"\nColumns ({len(eq_final.columns)}):")
print(eq_final.columns.tolist())
print(f"\nCategory distribution:")
print(eq_final['category'].value_counts())
print(f"\nMissing values summary:")
missing = eq_final.isnull().sum()
print(missing[missing > 0])

FINAL DATASET SUMMARY
Total records: 524,952
Date range: 1990-01-03 13:30:14 to 2025-11-20 16:51:56

Columns (43):
['rms', 'eventID', 'location', 'latitude', 'longitude', 'depth', 'type', 'magnitude', 'country', 'province', 'district', 'neighborhood', 'date', 'isEventUpdate', 'lastUpdateDate', 'category', 'year', 'month', 'day', 'hour', 'day_of_week', 'day_of_year', 'week_of_year', 'quarter', 'date_only', 'energy_joules', 'moment_dyne_cm', 'log_energy', 'log_moment', 'depth_class', 'magnitude_bin', 'fault_distance_km', 'province_std', 'soil_class', 'vs30', 'liquefaction_risk', 'seismic_hazard', 'moon_phase', 'illumination', 'moon_age', 'pressure_hpa', 'pressure_change', 'population_density']

Category distribution:
category
Tremor        520849
Earthquake      4103
Name: count, dtype: int64

Missing values summary:
location                  13
country                 2785
neighborhood           98590
lastUpdateDate        522872
fault_distance_km     520849
soil_class              9863

## 8. Export Processed Data

In [29]:
# Create processed data directory if it doesn't exist
os.makedirs(DATA_PROCESSED, exist_ok=True)

# Export full processed dataset
eq_final.to_csv(os.path.join(DATA_PROCESSED, 'earthquakes_processed.csv'), index=False)
print(f"Exported: earthquakes_processed.csv ({len(eq_final):,} records)")

# Export earthquakes only (M >= 4.0)
earthquakes_only = eq_final[eq_final['category'] == 'Earthquake']
earthquakes_only.to_csv(os.path.join(DATA_PROCESSED, 'earthquakes_only.csv'), index=False)
print(f"Exported: earthquakes_only.csv ({len(earthquakes_only):,} records)")

# Export tremors only (M < 4.0)
tremors_only = eq_final[eq_final['category'] == 'Tremor']
tremors_only.to_csv(os.path.join(DATA_PROCESSED, 'tremors_only.csv'), index=False)
print(f"Exported: tremors_only.csv ({len(tremors_only):,} records)")

Exported: earthquakes_processed.csv (524,952 records)
Exported: earthquakes_only.csv (4,103 records)


Exported: tremors_only.csv (520,849 records)


In [30]:
# Export summary statistics
summary_stats = eq_final.describe(include='all').T
summary_stats.to_csv(os.path.join(DATA_PROCESSED, 'summary_statistics.csv'))
print("Exported: summary_statistics.csv")

# Export yearly summary
yearly_summary = eq_final.groupby(['year', 'category']).agg({
    'eventID': 'count',
    'magnitude': ['mean', 'max'],
    'depth': 'mean',
    'energy_joules': 'sum'
}).round(2)
yearly_summary.columns = ['count', 'mag_mean', 'mag_max', 'depth_mean', 'total_energy']
yearly_summary.to_csv(os.path.join(DATA_PROCESSED, 'yearly_summary.csv'))
print("Exported: yearly_summary.csv")

Exported: summary_statistics.csv
Exported: yearly_summary.csv


In [31]:
print("\n" + "=" * 60)
print("DATA PREPROCESSING COMPLETE")
print("=" * 60)
print(f"\nProcessed files saved to: {DATA_PROCESSED}")
print("\nFiles created:")
for f in os.listdir(DATA_PROCESSED):
    fpath = os.path.join(DATA_PROCESSED, f)
    size = os.path.getsize(fpath) / (1024*1024)  # MB
    print(f"  - {f} ({size:.2f} MB)")


DATA PREPROCESSING COMPLETE

Processed files saved to: /Users/boraesen/Desktop/Semester 7/Stat495/stat495project/data/processed

Files created:
  - yearly_summary.csv (0.00 MB)
  - .DS_Store (0.01 MB)
  - eda_plots (0.00 MB)
  - earthquakes_only.csv (1.40 MB)
  - earthquakes_processed.csv (167.82 MB)
  - tremors_only.csv (166.42 MB)
  - earthquakes_clustered.csv (1.37 MB)
  - summary_statistics.csv (0.00 MB)
